In [ ]:
import os
import re
import shutil
import subprocess
from pathlib import Path

# === User settings ===
image_folder = Path("/home/disk/eos8/dglopez/pictures/Extreme_Event/CanESM5")  # <-- CHANGE THIS
output_video = "/home/disk/eos8/dglopez/pictures/Extreme_Event/Animations/CanESM5.mp4"
frame_rate = 1  # frames per second

# === Collect and sort files by number in filename ===
image_files = list(image_folder.glob("Most-Extreme-VPD_Upper_Day*.png"))

# Extract numeric part from filenames like model-20.png, model5.png
def extract_number(file):
    match = re.search(r'model(-?\d+)', file.stem)
    return int(match.group(1)) if match else float('inf')

# Sort files numerically
sorted_files = sorted(image_files, key=extract_number)

# === Create temporary folder with renamed frames ===
temp_folder = image_folder / "temp_ffmpeg_frames"
temp_folder.mkdir(exist_ok=True)

# Copy and rename to frame_000.png, frame_001.png, etc.
for i, file in enumerate(sorted_files):
    new_name = temp_folder / f"frame_{i:03d}.png"
    shutil.copy(file, new_name)

# === Build and run ffmpeg command ===
ffmpeg_cmd = [
    "ffmpeg",
    "-y",
    "-framerate", str(frame_rate),
    "-i", str(temp_folder / "frame_%03d.png"),
    "-c:v", "libx264",
    "-crf", "1",               # High quality
    "-pix_fmt", "yuv420p",
    str(output_video)
]

result = subprocess.run(ffmpeg_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# === Check result ===
if result.returncode == 0:
    print(f"✅ Video created: {output_video}")
    shutil.rmtree(temp_folder)  # Remove temporary folder
else:
    print("❌ FFmpeg failed:")
    print(result.stderr)


✅ Video created: /home/disk/eos8/dglopez/pictures/Extreme_Event/Animations/CanESM5.mp4
